In [1]:
import numpy as np
import arviz as az
import math
# import pymc3 as pm
import pickle
import matplotlib.pyplot as plt
from google.protobuf import text_format
from scipy.stats import multivariate_normal as mvn
from scipy.stats import skewnorm
from scipy.stats import norm
from scipy.interpolate import griddata
import pp_mix.protos.py.params_pb2 as params_pb2

from pp_mix.interface import ConditionalMCMC, cluster_estimate
from pp_mix.utils import loadChains, to_numpy, to_proto
from pp_mix.protos.py.state_pb2 import MultivariateMixtureState, EigenVector, EigenMatrix
from pp_mix.protos.py.params_pb2 import Params

In [2]:
import pandas as pd
with open("data/data_by_rule/p_50_d_2_M_4_nperclus_50_data.csv", newline='') as my_csv:
    data = pd.read_csv(my_csv, sep=',', header=None).values
    
print(data.shape)

(200, 50)


In [ ]:
p = 50
d = 2
M = 4
n_percluster_s = [50]

ntrick = 1000
nburn = 2000
niter = 5000
thin = 5

#for n_pc in n_percluster_s:
n_pc=50

print("Running p=50 d=2 M=4 nperclus={0}".format(n_pc))
with open("data/data_by_rule/p_50_d_2_M_4_nperclus_{0}_data.csv".format(n_pc), newline='') as my_csv:
    data = pd.read_csv(my_csv, sep=',', header=None).values

ranges = np.array([[-50,50],]*d).transpose()
ranges
sampler = ConditionalMCMC(params_file = "data/data_by_rule/params_compare_CD/sampler_params_p_50_d_2_M_4_n_{0}.asciipb".format(n_pc))
sampler.run(ntrick, nburn, niter, thin, data, ranges, 1)
sampler.serialize_chains("data/data_by_rule/chains_compare_CD/chains_p_50_d_2_M_4_nperclus_{0}.recordio".format(n_pc))
acc_rates = np.array([sampler.means_ar, sampler.lambda_ar])
np.savetxt("data/data_by_rule/chains_compare_CD/acc_rate_p_50_d_2_M_4_nperclus_{0}.csv".format(n_pc), acc_rates, delimiter=",")


Running p=50 d=2 M=4 nperclus=50
here we are
Number means in trick phase:  10
Trick, iter # 1  /  1000
Trick, iter # 2  /  1000
Trick, iter # 3  /  1000
Trick, iter # 4  /  1000
Trick, iter # 5  /  1000
Trick, iter # 6  /  1000
Trick, iter # 7  /  1000
Trick, iter # 8  /  1000
Trick, iter # 9  /  1000
Trick, iter # 10  /  1000
Trick, iter # 11  /  1000
Trick, iter # 12  /  1000
Trick, iter # 13  /  1000
Trick, iter # 14  /  1000
Trick, iter # 15  /  1000
Trick, iter # 16  /  1000
Trick, iter # 17  /  1000
Trick, iter # 18  /  1000
Trick, iter # 19  /  1000
Trick, iter # 20  /  1000
Trick, iter # 21  /  1000
Trick, iter # 22  /  1000
Trick, iter # 23  /  1000
Trick, iter # 24  /  1000
Trick, iter # 25  /  1000
Trick, iter # 26  /  1000
Trick, iter # 27  /  1000
Trick, iter # 28  /  1000
Trick, iter # 29  /  1000
Trick, iter # 30  /  1000
Trick, iter # 31  /  1000
Trick, iter # 32  /  1000
Trick, iter # 33  /  1000
Trick, iter # 34  /  1000
Trick, iter # 35  /  1000
Trick, iter # 36  /  

Trick, iter # 307  /  1000
Trick, iter # 308  /  1000
Trick, iter # 309  /  1000
Trick, iter # 310  /  1000
Trick, iter # 311  /  1000
Trick, iter # 312  /  1000
Trick, iter # 313  /  1000
Trick, iter # 314  /  1000
Trick, iter # 315  /  1000
Trick, iter # 316  /  1000
Trick, iter # 317  /  1000
Trick, iter # 318  /  1000
Trick, iter # 319  /  1000
Trick, iter # 320  /  1000
Trick, iter # 321  /  1000
Trick, iter # 322  /  1000
Trick, iter # 323  /  1000
Trick, iter # 324  /  1000
Trick, iter # 325  /  1000
Trick, iter # 326  /  1000
Trick, iter # 327  /  1000
Trick, iter # 328  /  1000
Trick, iter # 329  /  1000
Trick, iter # 330  /  1000
Trick, iter # 331  /  1000
Trick, iter # 332  /  1000
Trick, iter # 333  /  1000
Trick, iter # 334  /  1000
Trick, iter # 335  /  1000
Trick, iter # 336  /  1000
Trick, iter # 337  /  1000
Trick, iter # 338  /  1000
Trick, iter # 339  /  1000
Trick, iter # 340  /  1000
Trick, iter # 341  /  1000
Trick, iter # 342  /  1000
Trick, iter # 343  /  1000
T

Trick, iter # 616  /  1000
Trick, iter # 617  /  1000
Trick, iter # 618  /  1000
Trick, iter # 619  /  1000
Trick, iter # 620  /  1000
Trick, iter # 621  /  1000
Trick, iter # 622  /  1000
Trick, iter # 623  /  1000
Trick, iter # 624  /  1000
Trick, iter # 625  /  1000
Trick, iter # 626  /  1000
Trick, iter # 627  /  1000
Trick, iter # 628  /  1000
Trick, iter # 629  /  1000
Trick, iter # 630  /  1000
Trick, iter # 631  /  1000
Trick, iter # 632  /  1000
Trick, iter # 633  /  1000
Trick, iter # 634  /  1000
Trick, iter # 635  /  1000
Trick, iter # 636  /  1000
Trick, iter # 637  /  1000
Trick, iter # 638  /  1000
Trick, iter # 639  /  1000
Trick, iter # 640  /  1000
Trick, iter # 641  /  1000
Trick, iter # 642  /  1000
Trick, iter # 643  /  1000
Trick, iter # 644  /  1000
Trick, iter # 645  /  1000
Trick, iter # 646  /  1000
Trick, iter # 647  /  1000
Trick, iter # 648  /  1000
Trick, iter # 649  /  1000
Trick, iter # 650  /  1000
Trick, iter # 651  /  1000
Trick, iter # 652  /  1000
T

Trick, iter # 922  /  1000
Trick, iter # 923  /  1000
Trick, iter # 924  /  1000
Trick, iter # 925  /  1000
Trick, iter # 926  /  1000
Trick, iter # 927  /  1000
Trick, iter # 928  /  1000
Trick, iter # 929  /  1000
Trick, iter # 930  /  1000
Trick, iter # 931  /  1000
Trick, iter # 932  /  1000
Trick, iter # 933  /  1000
Trick, iter # 934  /  1000
Trick, iter # 935  /  1000
Trick, iter # 936  /  1000
Trick, iter # 937  /  1000
Trick, iter # 938  /  1000
Trick, iter # 939  /  1000
Trick, iter # 940  /  1000
Trick, iter # 941  /  1000
Trick, iter # 942  /  1000
Trick, iter # 943  /  1000
Trick, iter # 944  /  1000
Trick, iter # 945  /  1000
Trick, iter # 946  /  1000
Trick, iter # 947  /  1000
Trick, iter # 948  /  1000
Trick, iter # 949  /  1000
Trick, iter # 950  /  1000
Trick, iter # 951  /  1000
Trick, iter # 952  /  1000
Trick, iter # 953  /  1000
Trick, iter # 954  /  1000
Trick, iter # 955  /  1000
Trick, iter # 956  /  1000
Trick, iter # 957  /  1000
Trick, iter # 958  /  1000
T

Burnin, iter # 222  /  2000
Burnin, iter # 223  /  2000
Burnin, iter # 224  /  2000
Burnin, iter # 225  /  2000
Burnin, iter # 226  /  2000
Burnin, iter # 227  /  2000
Burnin, iter # 228  /  2000
Burnin, iter # 229  /  2000
Burnin, iter # 230  /  2000
Burnin, iter # 231  /  2000
Burnin, iter # 232  /  2000
Burnin, iter # 233  /  2000
Burnin, iter # 234  /  2000
Burnin, iter # 235  /  2000
Burnin, iter # 236  /  2000
Burnin, iter # 237  /  2000
Burnin, iter # 238  /  2000
Burnin, iter # 239  /  2000
Burnin, iter # 240  /  2000
Burnin, iter # 241  /  2000
Burnin, iter # 242  /  2000
Burnin, iter # 243  /  2000
Burnin, iter # 244  /  2000
Burnin, iter # 245  /  2000
Burnin, iter # 246  /  2000
Burnin, iter # 247  /  2000
Burnin, iter # 248  /  2000
Burnin, iter # 249  /  2000
Burnin, iter # 250  /  2000
Burnin, iter # 251  /  2000
Burnin, iter # 252  /  2000
Burnin, iter # 253  /  2000
Burnin, iter # 254  /  2000
Burnin, iter # 255  /  2000
Burnin, iter # 256  /  2000
Burnin, iter # 257  

In [8]:
sampler.run(ntrick, nburn, niter, thin, data, ranges, 1)

here we are
Number means in trick phase:  10
Trick, iter # 1  /  2
Trick, iter # 2  /  2
Burnin, iter # 1  /  2
Burnin, iter # 2  /  2
Running, iter # 1  /  2
Running, iter # 2  /  2
Allocated Means acceptance rate  0.83333333333333337034076748750521801412105560302734375
Lambda acceptance rate  0.1666666666666666574148081281236954964697360992431640625


In [ ]:
chain50 = loadChains("data/data_by_rule/chains_compare_CD/chains_p_50_d_2_M_4_nperclus_50.recordio", MultivariateMixtureState)
chain200 = loadChains("data/data_by_rule/chains_compare_CD/chains_p_50_d_2_M_4_nperclus_200.recordio", MultivariateMixtureState)


In [ ]:
nc_chain50 = np.array([x.ma for x in chain50])
nonall_50 = np.array([x.mna for x in chain50])

nc_chain200 = np.array([x.ma for x in chain200])
nonall_200 = np.array([x.mna for x in chain200])

print(nc_chain50.mean())
print(nonall_50.mean())

print(nc_chain200.mean())
print(nonall_200.mean())


In [ ]:
chain50[-1].clus_alloc
chain200[-1].clus_alloc

# check chain mixing on some variables 

In [ ]:
tau_chain50 = np.array([x.lamb_block.tau for x in chain50])
tau_chain200 = np.array([x.lamb_block.tau for x in chain200])

In [ ]:
plt.plot(tau_chain50)
plt.plot(tau_chain200)

In [ ]:
first_sigma_bar_chain50 = np.array([to_numpy(x.sigma_bar)[0] for x in chain50])
last_sigma_bar_chain50 = np.array([to_numpy(x.sigma_bar)[-1] for x in chain50])

first_sigma_bar_chain200 = np.array([to_numpy(x.sigma_bar)[0] for x in chain200])
last_sigma_bar_chain200 = np.array([to_numpy(x.sigma_bar)[-1] for x in chain200])

In [ ]:
fig_sigma = plt.figure()
ax1 = fig_sigma.add_subplot(221)
ax1.plot(first_sigma_bar_chain50)
plt.title('first sigmabar c50')

ax2 = fig_sigma.add_subplot(222)
ax2.plot(last_sigma_bar_chain50)
plt.title('last sigmabar c50')

ax3 = fig_sigma.add_subplot(223)
ax3.plot(first_sigma_bar_chain200)
plt.title('first sigmabar c200')

ax4 = fig_sigma.add_subplot(224)
ax4.plot(last_sigma_bar_chain200)
plt.title('last sigmabar c200')

plt.subplots_adjust(hspace=0.4)


# evaluate performance on simulate data: return performance dataframe 

In [ ]:
from scipy import stats
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from math import sqrt

p = 50
d = 2
M = 4
n_percluster_s = [50,200]
list_performance = list()


for n_percluster in n_percluster_s:
    chain = loadChains("data/data_script_sim/chains/chains_p_{0}_d_{1}_M_{2}_nperclus_{3}.recordio".format(p,d,M,n_percluster), MultivariateMixtureState)
    acc_rates = np.loadtxt("data/data_script_sim/chains/acc_rate_p_{0}_d_{1}_M_{2}_nperclus_{3}.csv".format(p,d,M,n_percluster), delimiter=',')

    n_cluster_chain = np.array([x.ma for x in chain])
    post_mode_nclus = stats.mode(n_cluster_chain)[0][0] # store in dataframe
    post_avg_nclus = n_cluster_chain.mean() # store in dataframe
    
    clus_alloc_chain = [x.clus_alloc for x in chain]
    best_clus = cluster_estimate(np.array(clus_alloc_chain))
    true_clus = np.repeat(range(M),n_percluster)
    ari_best_clus = adjusted_rand_score(true_clus, best_clus) # store in dataframe
    
    aris_chain = np.array([adjusted_rand_score(true_clus, x) for x in clus_alloc_chain])
    mean_aris, sigma_aris = np.mean(aris_chain), np.std(aris_chain) # store mean_aris in dataframe
    CI_aris = stats.norm.interval(0.95, loc=mean_aris, scale=sigma_aris/sqrt(len(aris_chain))) # store in dataframe
    
    list_performance.append([p,d,M,n_percluster,acc_rates[0],acc_rates[1],post_mode_nclus,
                        post_avg_nclus,ari_best_clus,mean_aris,CI_aris])
    

    
df_performance = pd.DataFrame(list_performance, columns=('dim_p', 'dim_d', 'dim_M', 'dim_n_pc', 'means_ar','lambda_ar',
                                      'mode_nclus', 'avg_nclus', 'ari_best_clus', 'mean_ari', 'CI_ari'))
    
